In [1]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 59.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 27.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 125.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 73.3 MB/s eta 0:00:00


In [2]:
import pandas as pd
import numpy as np
import torch
from torch import nn
from torch.utils.data import DataLoader,Dataset
import json
import argparse
from transformers import AutoTokenizer
from  collections import  OrderedDict
import torch
from torch.utils.data import DataLoader,Dataset
from torch.nn import functional as F
from sklearn.metrics import  accuracy_score

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
path="drive/MyDrive/NLP_2023/HW2"


In [5]:
train_data=f"{path}/data/coarse-grained/train_coarse_grained.json"
test_data=f"{path}/data/coarse-grained/test_coarse_grained.json"
map_data=f"{path}/data/map/coarse_fine_defs_map.json"

In [6]:

def read_map_data(json_file):
    with open(json_file) as json_file:
        df = json.load(json_file)
        dict_map=OrderedDict()
        for key in df.keys():
            l=[]
            for item in df[key]:
                key1,value=list(item.items())[0]
                l.append(value)
            dict_map[key]=l
        return dict_map


mapper = read_map_data( map_data)


In [7]:
def read_files(json_file):
    with open(json_file) as json_file:
        df = json.load(json_file)
        l=[]
        for key in df.keys():
            instance_ids=[x for x in df[key]["instance_ids"].keys()]
            for i,ID in enumerate(instance_ids):
                instance_id=df[key]["instance_ids"][ID]
                lemmas=df[key]["lemmas"]
                pos_tags=df[key]["pos_tags"]
                senses_key=int(list(df[key]["senses"].keys())[i])
                senses_words=df[key]["senses"][str(senses_key)][0]
                words=df[key]["words"]
                candidates_key=int(list(df[key]["candidates"].keys())[i])
                candidates_words=df[key]["candidates"][str(candidates_key)]
                l.append([instance_id,lemmas,pos_tags,senses_key,senses_words,words,candidates_key,candidates_words])
        df=pd.DataFrame(l,columns=["instance_id","lemmas","pos_tags","senses_keys","senses_words","words",
                 "candidates_key","candidates_words"])
        return(df)

In [8]:
train_df=read_files(train_data)
test_df=read_files(test_data)


train_df["word"]=train_df.apply(lambda x : x["lemmas"][x["candidates_key"]] , axis=1)
test_df["word"]=test_df.apply(lambda x : x["lemmas"][x["candidates_key"]] , axis=1)




tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")
train_df["tokenizer"]=train_df["lemmas"].apply(lambda x : tokenizer.convert_tokens_to_ids(x))
test_df["tokenizer"]=test_df["lemmas"].apply(lambda x : tokenizer.convert_tokens_to_ids(x))

In [9]:
def get_word_2_indices(encoded, tokenizer):

    # we have to collect all the subtokenizations of the words
    w_id2input_ids = dict()
    # we have to keep track of the indices that compose the word
    # so we can use them to extract the embeddings
    w_id2_indices = dict()
    for idx, (w_id, input_id) in enumerate(zip(encoded.word_ids(), encoded.input_ids[0])):
        w_id2input_ids.setdefault(w_id, []).append(input_id)
        w_id2_indices.setdefault(w_id, []).append(idx)

    # now let's decode all the words that we have
    word2_indices = dict()
    for w_id, input_ids in w_id2input_ids.items():
        word = tokenizer.decode(input_ids)
        word2_indices[word] = w_id2_indices[w_id]
    return word2_indices

# produce word vectors by averaging
# all embeddings of the bpes of the word
def get_word_vector1(sent, word, tokenizer, model):
    # encode_plus returns additional information such as the word ids.
    encoded = tokenizer.encode_plus(sent, return_tensors="pt")
    # to guarantee the tensors are on the same device as the model
    encoded = encoded.to(model.device)
    # get the mapping between the tokens and the indices of the embeddings
    word2_indices = get_word_2_indices(encoded, tokenizer)
    # with torch.no_grad():
    output = model(**encoded)
    # we get the word embeddings of the last layer of the last hidden state
    last_hidden_state = output[0]
    # bert computes embeddings in batches we gave a single sentence
    # so we have just one element in the batch and can extract the token embeddings
    sentence_token_embeddings = last_hidden_state[0]

    # we take the embeddings correspondings to the indices of the BPEs that compose our word
    word_bpes_embeddings = sentence_token_embeddings[word2_indices[word]]
    # word_bpes_embeddings is a 2D-shaped matrix. dim=0 is needed to specify the axis to compute the mean along.
    # In this case, dim=0 is chosen since we want to average all the row vectors, which correspond to bpe embeddings.
    return word_bpes_embeddings.mean(dim=1)



def get_word_vector2(sent, tokenizer, model):
    # encode_plus returns additional information such as the word ids.
    encoded = tokenizer.encode_plus(sent, return_tensors="pt")
    # to guarantee the tensors are on the same device as the model
    encoded = encoded.to(model.device)
    # get the mapping between the tokens and the indices of the embeddings
    # with torch.no_grad():
    output = model(**encoded)
    # we get the word embeddings of the last layer of the last hidden state
    last_hidden_state = output[0]
    # bert computes embeddings in batches we gave a single sentence
    # so we have just one element in the batch and can extract the token embeddings
    sentence_token_embeddings = last_hidden_state[0]
    return sentence_token_embeddings.mean(dim=1)


def get_transformers_similarity(tokenizer, model, s1, s2, target_token_s1, target_token_s2=None):
    word_embedding_s1 = get_word_vector1(s1, target_token_s1, tokenizer, model)
    if target_token_s2 :
        word_embedding_s2 = get_word_vector1(s2, target_token_s2, tokenizer, model)
    else:
        word_embedding_s2 = get_word_vector2(s2, tokenizer, model)
    return torch.cosine_similarity(word_embedding_s1.unsqueeze(dim=0), word_embedding_s2.unsqueeze(dim=0))



def model_similarty(candidates,txt,candidate_key,senses_words,model):
    sent=' '.join(word for word in txt)
    candidate_sents=OrderedDict({candidate:mapper[candidate] for candidate in candidates if  candidate in mapper.keys()})
    target_token_1=txt[candidate_key]
    try :
        scores=[]
        keys=[]
        for  key in candidate_sents.keys():
            score=[]
            for s2 in candidate_sents[key]:
                score.append(get_transformers_similarity(tokenizer, model, sent ,s2 ,target_token_1))
            scores.append(max(score))
            keys.append(key)
        scores=torch.softmax(torch.stack(scores),0)
        scores=np.hstack((scores.squeeze().detach().numpy(),np.zeros(15-len(scores))))
        return keys,scores
    except:
        return None


In [10]:
from transformers import AutoTokenizer, AutoModel

checkpoint = "bert-base-cased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModel.from_pretrained(checkpoint)


Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
train_df["param"]=train_df.apply(lambda x : model_similarty(x["candidates_words"],x["lemmas"],
                                                            x["candidates_key"],x["senses_words"],model) ,axis=1)

train_df.dropna(inplace=True )
train_df["model_pred"]=train_df["param"].apply(lambda x : x[1])
train_df["C_words"]=train_df["param"].apply(lambda x : x[0])
train_df.reset_index(inplace=True , drop=True)
train_df.to_pickle(f"{path}/data/train_df.pkl")

In [ ]:
test_df["param"]=test_df.apply(lambda x : model_similarty(x["candidates_words"],x["lemmas"],
                                                            x["candidates_key"],x["senses_words"],model) ,axis=1)

test_df.dropna(inplace=True )
test_df["model_pred"]=test_df["param"].apply(lambda x : x[1])
test_df["C_words"]=test_df["param"].apply(lambda x : x[0])
test_df.reset_index(inplace=True , drop=True)
test_df.to_pickle(f"{path}/data/test_df.pkl")

In [6]:
train_df=pd.read_pickle(f"{path}/data/train_df.pkl")
test_df=pd.read_pickle(f"{path}/data/test_df.pkl")

In [7]:
class dataset(Dataset):
    def __init__(self,dataframe):
        super().__init__()
        self.df=dataframe


    def __len__(self):
        return len(self.df)

    def __getitem__(self,index):
        pred,candidates,senses_word=self.df.iloc[index][["model_pred","C_words","senses_words"]]
        lbl=candidates.index(senses_word)
        lbl=F.one_hot(torch.tensor(lbl),15)
        return torch.tensor(pred).float(),lbl


In [8]:
T_model = nn.Sequential(OrderedDict([
                                      ('fc1', nn.Linear(15,50)),
                                       ('relu1', nn.ReLU()),
                                       ('fc2', nn.Linear(50,15)),
                                      ('relu2', nn.Softmax(dim=0))
                                    ]))




train_ds=dataset(train_df)
train_dl=DataLoader(train_ds,batch_size=64)

In [9]:
epochs=5

optimizer=torch.optim.Adam(T_model.parameters(),lr=1e-3)
criterion=nn.CrossEntropyLoss()
total_step=len(train_dl)
for epoch in range(epochs):
    for step,(inputs,lbls) in enumerate(train_dl):
        train_pred=T_model(inputs)
        loss=criterion(lbls.float(),train_pred)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        pred=torch.argmax(train_pred,1)
        lbls=torch.argmax(lbls,1)
        correct = (pred == lbls).sum().item()
        acc=100*(correct/len(train_pred))
        if (step+1) % 20 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f} , accuracy: {:.2f}'
                    .format(epoch+1, epochs+1, step+1, total_step, loss.item(),acc))




Epoch [1/6], Step [20/268], Loss: 0.6442 , accuracy: 3.12
Epoch [1/6], Step [40/268], Loss: 0.6437 , accuracy: 70.31
Epoch [1/6], Step [60/268], Loss: 0.6435 , accuracy: 54.69
Epoch [1/6], Step [80/268], Loss: 0.6423 , accuracy: 67.19
Epoch [1/6], Step [100/268], Loss: 0.6414 , accuracy: 68.75
Epoch [1/6], Step [120/268], Loss: 0.6406 , accuracy: 60.94
Epoch [1/6], Step [140/268], Loss: 0.6398 , accuracy: 67.19
Epoch [1/6], Step [160/268], Loss: 0.6379 , accuracy: 65.62
Epoch [1/6], Step [180/268], Loss: 0.6383 , accuracy: 67.19
Epoch [1/6], Step [200/268], Loss: 0.6361 , accuracy: 60.94
Epoch [1/6], Step [220/268], Loss: 0.6348 , accuracy: 59.38
Epoch [1/6], Step [240/268], Loss: 0.6341 , accuracy: 64.06
Epoch [1/6], Step [260/268], Loss: 0.6358 , accuracy: 62.50
Epoch [2/6], Step [20/268], Loss: 0.6325 , accuracy: 71.88
Epoch [2/6], Step [40/268], Loss: 0.6305 , accuracy: 70.31
Epoch [2/6], Step [60/268], Loss: 0.6346 , accuracy: 56.25
Epoch [2/6], Step [80/268], Loss: 0.6295 , accur

In [10]:
test_ds=dataset(test_df)
from tqdm import tqdm
t_pred=[]
t_lbl=[]
for index in tqdm(range(len(test_df))):
    test_inputs,test_lbls =test_ds.__getitem__(index)
    test_pred=T_model(test_inputs)
    rt_pred=torch.argmax(test_pred).item()
    rt_lbl=torch.argmax(test_lbls).item()

    t_pred.append(rt_pred)
    t_lbl.append(rt_lbl)
test_acc=accuracy_score(t_lbl,t_pred)
print (f"test accuracy : {test_acc}")

100%|██████████| 949/949 [00:00<00:00, 1125.91it/s]

test accuracy : 0.6859852476290832
